# Projekt Machine Learning

## Das Kalman Filter

In [2]:
from DataGenerationRadar3D import *
from DBScan import *
from ui import interactive1DExperiment, interactiveDBScan, interactive3DExperiment
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy
from collections import deque
from ipywidgets import *
from collections import deque
%matplotlib widget

In [5]:
class KalmanFilter:
    # Initialisierung von Kalman Filter
    def __init__(self, s_hat, transition_model, H, Q, R):
        self.s_hat = s_hat
        self.P_hat = np.eye(len(s_hat)) * 100
        self.model = transition_model
        self.H = H # Measurement Function
        self.Q = Q # Process Noise
        self.R = R # Measurement Noise
    
    # Kalman Filter Algorithmus
    def step(self, z):
        # Prediction
        s_hat_p = self.model @ self.s_hat
        P_hat_p = self.model @ self.P_hat @ self.model.T + self.Q
        # Calculate Kalman Gain
        K = P_hat_p @ self.H.T @ np.linalg.inv(self.H @ P_hat_p @ self.H.T + self.R)
        # Update covariance of estimation error
        self.P_hat = self.P_hat - K @ self.H @ self.P_hat
        # Improve estimate
        e_m_p = z - self.H @ s_hat_p
        self.s_hat = s_hat_p + K @ e_m_p
        return self.s_hat

## 1D Experiment

## 3D Experiment (ohne DBSCAN)

## DBSCAN

In [4]:
def pairwise_sq_distance(X1, X2):
    # Calculate the pairwise distance between all pairs of points from X1 and X2. 
    return np.sum(X1**2, axis=1, keepdims=True) - 2*np.matmul(X1, X2.T) + np.sum(X2**2, axis=1, keepdims=True).T

class DBSCAN():
    
    def __init__(self, eps=0.5, minpts=5):
        self.eps = eps
        self.minpts = minpts
        
    def fit(self, X):
        dist = pairwise_sq_distance(X, X)
        neighbours = list(map(lambda d: np.arange(d.shape[0])[d < self.eps**2], dist))
        
        # Label all points as outliers initially.
        self.assignment = np.full((X.shape[0],), -1, dtype=int)
        # Find core points.
        ## Determine the number of neighbors of each point.
        N_neighbors = np.sum(dist < self.eps**2, axis=1)
        self.assignment[N_neighbors >= self.minpts] = -2
        
        # Create clusters.
        cluster = 0
        stack = deque()
        for p in range(X.shape[0]):
            if self.assignment[p] != -2:
                continue
                
            self.assignment[p] = cluster 
            
            stack.extend(neighbours[p])
            # Expand cluster outwards. 
            while len(stack) > 0:
                n = stack.pop()
                label = self.assignment[n]
                # If core point include all points in ε-neighborhood.
                if label == -2:
                    stack.extend(neighbours[n])
                # If not core point (edge of cluster).
                if label < 0:
                    self.assignment[n] = cluster
            
            cluster += 1
            
    def fit_predict(self, X):
        self.fit(X)
        return self.assignment
    
    def predict(self,X):
        return self.assignment